In [4]:
#ML_result_seed_analysis         23.07.05  -> 24.04.18
#To find how the ML performance fluctuates based on seeds.

import pandas as pd
import os

os.getcwd()

'/Users/m221138/RA_ACPA_multiomics/src/addressing_feedback/machine_learning_handle_imbalanced'

In [5]:
def make_max_acc_dict(data_df, i, max_acc_dict, cutoff_list):
    
    df_column_name_list = ['ACC_average', 'ACC_stdev', 'PRE_average','PRE_stdev', 
                           'TPR_average','TPR_stdev', 'TNR_average','TNR_stdev',  
                           'FPR_average','FPR_stdev', 
                           'FNR_average','FNR_stdev', 
                           'NPV_average','NPV_stdev']
    
    seed = "seed_%s" % i
    r, c = data_df.shape
    
    for i in range(r):
        model = data_df["Algorithm"][i]     
        cutoff = data_df["cutoff"][i]
        if cutoff not in cutoff_list:
            cutoff_list.append(cutoff)
        
        for df_column_name in df_column_name_list:
            max_acc_dict[seed, model, cutoff, df_column_name] = data_df[df_column_name][i]

    return max_acc_dict, cutoff_list

def main(data_dir, classification_type_list):
    #buffer
    df_column_name_list = ['ACC_average', 'ACC_stdev', 'PRE_average','PRE_stdev', 
                           'TPR_average','TPR_stdev', 'TNR_average','TNR_stdev',  
                           'FPR_average','FPR_stdev', 
                           'FNR_average','FNR_stdev', 
                           'NPV_average','NPV_stdev']
    
    omics_type_list = ["1omics_M"]
    classifier_type_list = ["RF"]
    
    max_acc_dict = {}
    cutoff_list = []
    batch_dict = {'batch_0_500': [0,500]}
    
    for batch_num in list(batch_dict.keys()):
        for i in range(batch_dict[batch_num][0], batch_dict[batch_num][1]+1):        
            data_file = "%s/%s/seed_%s/ml_summary.tsv" % (data_dir, batch_num, i)
            data_df = pd.read_csv(data_file, sep="\t")
            max_acc_dict, cutoff_list = make_max_acc_dict(data_df, i, max_acc_dict, cutoff_list)
            
    #summary
    output_file = "%s/batch_summary/ML_seed_results.tsv" % data_dir
    output_txt = open(output_file, 'w')
    
    #header
    output_txt.write("seed\tclassifier\tcutoff")
    for df_column_name in df_column_name_list:
        output_txt.write("\t%s" % df_column_name)
    output_txt.write("\n")
    
    #main
    for batch_num in list(batch_dict.keys()):
        for i in range(batch_dict[batch_num][0], batch_dict[batch_num][1]+1):
            seed = 'seed_%s' % i
            for cutoff in cutoff_list:
                output_txt.write("%s\t%s" % (seed, "RF"))
                output_txt.write("\t%s" %  cutoff)
                for df_column_name in df_column_name_list:
                    output_txt.write('\t%s' % max_acc_dict[seed, "RF", cutoff, df_column_name])
                output_txt.write('\n')
    
    output_txt.close()

In [6]:
data_dir = "../../../analysis_addressing_feedback/external_validation_handle_imbalanced/ra_mtx_metabolomics"
main(data_dir, ["posVSneg"])

In [8]:
data_dir = "../../../analysis_addressing_feedback/external_validation_handle_imbalanced/ra_plasma_metabolomics"
main(data_dir, ["posVSneg"])